In [1]:
# Importing the required libraries
import cv2
import time
import numpy as np
import pandas as pd

### Download pose_iter_440000.caffemodel weightsfile from [here](http://posefs1.perception.cs.cmu.edu/OpenPose/models/pose/mpi/pose_iter_160000.caffemodel).

In [5]:
# COCO is a body pose estimation model

protoFile = "../pose/coco/pose_deploy_linevec.prototxt"
weightsFile = "../pose/coco/pose_iter_440000.caffemodel"
nPoints = 18

# Creating list of pairs of coordinates to join for pose detection.
POSE_PAIRS = [ [1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]


In [6]:
# Initializing variables to change video frames resolution
inWidth = 368
inHeight = 368
threshold = 0.1

# Reading sample video using OpenCV VideoCapture
input_source = "sample_video.mp4"
cap = cv2.VideoCapture(input_source)
hasFrame, frame = cap.read()

In [7]:
# Creating object to write frames to 'output.avi' video
vid_writer = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1],frame.shape[0]))

# Reading network model stored in caffemodel
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

In [8]:
# Creating dataframe to store generated coordinates
df = pd.DataFrame(columns = range(1,1751))

In [10]:
# Iterating through all the frames

for ij in range(1,1751):
    print(ij)
    t = time.time()
    hasFrame, frame = cap.read()
    frameCopy = np.copy(frame)
    if not hasFrame:
        cv2.waitKey()
        break

    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    
    # Mean subtraction and Scaling on frames
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                              (0, 0, 0), swapRB=False, crop=False)
    net.setInput(inpBlob)
    output = net.forward()

    H = output.shape[2]
    W = output.shape[3]

    # Empty list to store the detected keypoints
    points = []

    for i in range(nPoints):
        # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]

        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
        
        # Scale the point to fit on the original image
        x = (frameWidth * point[0]) / W
        y = (frameHeight * point[1]) / H

        if prob > threshold : 
            cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)

            # Add the point to the list if the probability is greater than the threshold
            points.append((int(x), int(y)))
        else :
            points.append(None)
    
    # Storing coordinates in dataframe
    series = pd.Series(points)
    df[ij] = series
    
    # Draw Skeleton
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            cv2.line(frame, points[partA], points[partB], (0, 255, 255), 3, lineType=cv2.LINE_AA)
            cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.circle(frame, points[partB], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

    #Writing time elapsed on video frame
    cv2.putText(frame, "time taken = {:.2f} sec".format(time.time() - t), (50, 50), cv2.FONT_HERSHEY_COMPLEX, .8, (255, 50, 0), 2, lineType=cv2.LINE_AA)
    cv2.imshow('Output-Skeleton', frame)
    
    #Writing to the vid_writer object
    vid_writer.write(frame)

# Saving on your system
vid_writer.release()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [12]:
df.head()

,1,2,3,4,5,6,7,8,9,10,...,1740,1741,1742,1743,1744,1745,1746,1747,1748,1749
0,"(262, 156)","(262, 156)","(262, 156)","(262, 172)","(262, 172)","(262, 156)","(262, 172)","(262, 156)","(275, 156)","(275, 156)",...,"(200, 78)","(212, 62)","(225, 62)","(225, 62)","(237, 62)","(250, 62)","(250, 62)","(262, 46)","(262, 46)","(262, 46)"
1,"(288, 234)","(288, 234)","(288, 234)","(288, 250)","(300, 250)","(300, 250)","(300, 250)","(300, 250)","(300, 234)","(300, 234)",...,"(237, 140)","(250, 140)","(262, 140)","(275, 140)","(275, 140)","(275, 140)","(288, 140)","(288, 140)","(288, 140)","(288, 140)"
2,"(250, 234)","(250, 234)","(237, 234)","(237, 250)","(250, 250)","(250, 250)","(250, 250)","(250, 250)","(262, 250)","(262, 250)",...,"(212, 156)","(225, 156)","(237, 156)","(237, 156)","(237, 156)","(250, 156)","(250, 156)","(237, 140)","(250, 140)","(250, 140)"
3,"(175, 234)","(187, 234)","(175, 250)","(187, 266)","(187, 297)","(187, 313)","(212, 328)","(237, 328)","(250, 328)","(262, 328)",...,"(162, 234)","(162, 234)","(175, 234)","(187, 234)","(187, 250)","(187, 250)","(187, 250)","(187, 250)","(200, 234)","(200, 234)"
4,"(162, 266)","(162, 250)","(162, 250)","(175, 266)","(175, 250)","(175, 266)","(175, 266)","(187, 281)","(200, 313)","(200, 313)",...,"(200, 234)","(212, 234)","(225, 234)","(225, 234)","(237, 250)","(237, 250)","(250, 250)","(250, 250)","(262, 250)","(262, 250)"


In [14]:
#Writing to the csv file
df.to_csv('coordinates.csv', index = None)